In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(400_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0324 22:42:23.910924295    3110 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0324 22:42:31.071912268    3110 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [3]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [4]:
# ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']
ran_operators = []

In [5]:
pbar = tqdm()

0it [00:00, ?it/s]

In [9]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in [290]:
    not_ran_operators = []
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    if agency == 278:
        analysis_date = dt.date(2022, 2, 17)  ##one-off sdmts
    else:
        analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables
    day = str(analysis_date.day).zfill(2)
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
        shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
                                                 f'{utils.GCS_FILE_PATH}segment_speed_views',
                                                f'{agency}_02_{day}_pm_peak'
                                                )
        rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        # m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
        # rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        delay_to_parquet.to_parquet(f'{utils.GCS_FILE_PATH}endpoint_delay_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 290...
found parquet
found parquet
found parquet
found parquet
found_parquet
11402517-137-Blocks-Weekday-51
11403326-137-Blocks-Weekday-14
11401834-137-Blocks-Weekday-51
11402478-137-Blocks-Weekday-51
11427096-137-Blocks-Weekday-14
11403562-137-Blocks-Weekday-14
11403225-137-Blocks-Weekday-14
11403590-137-Blocks-Weekday-51
11427350-137-Blocks-Weekday-51
11427080-137-Blocks-Weekday-14
11427081-137-Blocks-Weekday-51
11402028-137-Blocks-Weekday-51
11402688-137-Blocks-Weekday-14
11401868-137-Blocks-Weekday-14
11401851-137-Blocks-Weekday-51
11402770-137-Blocks-Weekday-51
11403182-137-Blocks-Weekday-14
11402784-137-Blocks-Weekday-14
11402692-137-Blocks-Weekday-14
11401987-137-Blocks-Weekday-51
11402729-137-Blocks-Weekday-51
11403096-137-Blocks-Weekday-14
11402002-137-Blocks-Weekday-14
11402503-137-Blocks-Weekday-14
11402014-137-Blocks-Weekday-14
11427339-137-Blocks-Weekday-14
11403298-137-Blocks-Weekday-51
11401925-137-Blocks-Weekday-14
11454057-137-Blocks-Weekday-51


In [8]:
agency = 208
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [9]:
rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
rt_day.set_filter(start_time='15:00', end_time='19:00')

found parquet
found parquet
found parquet
found parquet
found_parquet
99340001
99440001
99540001
7540002
7530503
99140001
7530506
7530504
99240001
2440011
2440015
2440018
2440014
2440010
2440022
2440019
2440026
2440027
2440032
2440030
2440034
2440035
2440023
2440033
2440031
9440003
9440005
9440004
9440001
9440002
9440008
9440006
9440007
2440025
2440028
2440024
2440029
2440021
9540002
9540004
9540003
9540005
9540001
9540007
9540006
9540008
9340004
9340006
9340005
340001
340003
340004
340002
9340003
340007
2440005
340006
9340007
9340008
340005
2440007
9340009
9340010
340009
340008
340010
4740009
4740012
4740013
4740011
4740004
4740001
4740014
4740010
4740005
4740006
4740007
4740018
4740003
4740008
4740015
4740002
4740016
4740017
4740021
4740023
4740019
4740022
4740020
4740024
6140005
6140007
6140006
6140003
6140001
6140004
6140010
6140002
6140008
6140009
6140011
6140012
9140003
9240002
140003
9240001
9140001
140005
9140004
9140002
9240003
140008
9140005
9340002
9240004
9240005
2440013
24

ValueError: Expected a 1D array, got an array with shape (0, 10)

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
                                         f'{utils.GCS_FILE_PATH}segment_speed_views',
                                        f'{agency}_02_{day}_pm_peak'
                                        )
rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
# m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
# rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
delay_to_parquet = rt_day.endpoint_delay_view.copy()
delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
delay_to_parquet = delay_to_parquet >> select(-_.delay)
delay_to_parquet.to_parquet(f'{utils.GCS_FILE_PATH}endpoint_delay_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling

In [7]:
# rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling

### Ongoing issues
* 358 Union City too many trips without shape?

In [8]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')